In [1]:
import numpy as np
import pandas as pd
from datetime import date

import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
plt.rcParams['font.family'] = 'IPAPGothic' #全体のフォントを設定
plt.rcParams['font.size'] = 12 #フォントサイズを設定 default : 12

In [3]:
CSV_DIR = "csv"
TARGET_YEAR = 2018

In [4]:
df = pd.read_csv(f'{CSV_DIR}/horse_data.{TARGET_YEAR}.csv', index_col=[0])

In [5]:
df.drop(['owner_id', 'birth_year', 'owner'], axis=1, inplace=True)

In [6]:
df.head()

,id,name,sex,stable,trainer_id,sire,mare,bms,breeder,breeder_id,prize
0,2015105046,ラッキーライラック,牝,松永幹夫,01092,オルフェーヴル,ライラックスアンドレース,Flower Alley,ノーザンファーム,373126,20460.6
1,2015104961,アーモンドアイ,牝,国枝栄,00399,ロードカナロア,フサイチパンドラ,サンデーサイレンス,ノーザンファーム,373126,17595.8
2,2015102440,エポカドーロ,牡,藤原英昭,01055,オルフェーヴル,ダイワパッション,フォーティナイナー,田上徹,810098,17421.2
3,2015102367,ダノンプレミアム,牡,中内田充,01137,ディープインパクト,インディアナギャル,Intikhab,ケイアイファーム,253025,16651.3
4,2015105064,ステルヴィオ,牡,木村哲也,01126,ロードカナロア,ラルケット,ファルブラヴ,ノーザンファーム,373126,13669.0


In [7]:
df_add = pd.read_csv(f'{CSV_DIR}/horse_data_2.{TARGET_YEAR}.csv', parse_dates=["birth_date"], index_col=[0])

In [8]:
df_add.head()

,id,name,hair,birth_date,race_result,debut_weight,sales_price,relatives
0,2015105046,ラッキーライラック,栗毛,2015-04-03,4-1-0-0,480.0,NaN,ラルク、ライラックローズ
1,2015104961,アーモンドアイ,鹿毛,2015-03-10,3-1-0-0,472.0,NaN,スペルヴィア、ピュクシス
2,2015102440,エポカドーロ,黒鹿毛,2015-02-15,3-1-1-0,484.0,NaN,ダイワインスパイア、ダイワフェーム
3,2015102367,ダノンプレミアム,青鹿毛,2015-04-03,4-0-0-0,476.0,NaN,ロードプレミアム、ロードグランツ
4,2015105064,ステルヴィオ,鹿毛,2015-01-15,3-2-0-1,462.0,NaN,ボルゲーゼ、カルナローリ


In [9]:
df_add['race_count'] = df_add['race_result'].map(lambda x: sum([int(cnt) for cnt in x.split('-')]))

In [10]:
df_add['win_count'] = df_add['race_result'].map(lambda x: int(x.split('-')[0]))

In [11]:
df_add['birth_date_from_beginning_of_year'] = df_add['birth_date'].map(lambda x:(x.date() - date(TARGET_YEAR - 3, 1, 1)).days).astype(int)

In [12]:
df_add.drop(['race_result', 'birth_date', 'sales_price', 'relatives'], inplace=True, axis=1)

In [13]:
df_add.head()

,id,name,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year
0,2015105046,ラッキーライラック,栗毛,480.0,5,4,92
1,2015104961,アーモンドアイ,鹿毛,472.0,4,3,68
2,2015102440,エポカドーロ,黒鹿毛,484.0,5,3,45
3,2015102367,ダノンプレミアム,青鹿毛,476.0,4,4,92
4,2015105064,ステルヴィオ,鹿毛,462.0,6,3,14


In [14]:
df = pd.merge(df, df_add, on='id', how='left', suffixes=('', '_2'))

In [15]:
df.head()

,id,name,sex,stable,trainer_id,sire,mare,bms,breeder,breeder_id,prize,name_2,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year
0,2015105046,ラッキーライラック,牝,松永幹夫,01092,オルフェーヴル,ライラックスアンドレース,Flower Alley,ノーザンファーム,373126,20460.6,ラッキーライラック,栗毛,480.0,5,4,92
1,2015104961,アーモンドアイ,牝,国枝栄,00399,ロードカナロア,フサイチパンドラ,サンデーサイレンス,ノーザンファーム,373126,17595.8,アーモンドアイ,鹿毛,472.0,4,3,68
2,2015102440,エポカドーロ,牡,藤原英昭,01055,オルフェーヴル,ダイワパッション,フォーティナイナー,田上徹,810098,17421.2,エポカドーロ,黒鹿毛,484.0,5,3,45
3,2015102367,ダノンプレミアム,牡,中内田充,01137,ディープインパクト,インディアナギャル,Intikhab,ケイアイファーム,253025,16651.3,ダノンプレミアム,青鹿毛,476.0,4,4,92
4,2015105064,ステルヴィオ,牡,木村哲也,01126,ロードカナロア,ラルケット,ファルブラヴ,ノーザンファーム,373126,13669.0,ステルヴィオ,鹿毛,462.0,6,3,14


In [16]:
df.drop(['id', 'name_2'], axis=1, inplace=True)

In [17]:
df.head()

,name,sex,stable,trainer_id,sire,mare,bms,breeder,breeder_id,prize,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year
0,ラッキーライラック,牝,松永幹夫,01092,オルフェーヴル,ライラックスアンドレース,Flower Alley,ノーザンファーム,373126,20460.6,栗毛,480.0,5,4,92
1,アーモンドアイ,牝,国枝栄,00399,ロードカナロア,フサイチパンドラ,サンデーサイレンス,ノーザンファーム,373126,17595.8,鹿毛,472.0,4,3,68
2,エポカドーロ,牡,藤原英昭,01055,オルフェーヴル,ダイワパッション,フォーティナイナー,田上徹,810098,17421.2,黒鹿毛,484.0,5,3,45
3,ダノンプレミアム,牡,中内田充,01137,ディープインパクト,インディアナギャル,Intikhab,ケイアイファーム,253025,16651.3,青鹿毛,476.0,4,4,92
4,ステルヴィオ,牡,木村哲也,01126,ロードカナロア,ラルケット,ファルブラヴ,ノーザンファーム,373126,13669.0,鹿毛,462.0,6,3,14


In [18]:
df_mare = pd.read_csv(f'{CSV_DIR}/horse_data_mare.{TARGET_YEAR}.csv', index_col=[0])

In [19]:
df_mare.head()

,id,name,crop_count,crop_win_count,crop_grade_horse_count,crop_grade_win_count
0,000a011df8,ライラックスアンドレース,5,7,1,3
1,2003103043,フサイチパンドラ,9,10,1,2
2,2003101283,ダイワパッション,7,11,1,1
3,000a011a94,インディアナギャル,6,7,1,3
4,2005102452,ラルケット,4,5,1,1


In [20]:
# 簡易nullチェック
df_mare.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5692 entries, 0 to 5691
Data columns (total 6 columns):
id                        5692 non-null object
name                      5692 non-null object
crop_count                5692 non-null int64
crop_win_count            5692 non-null int64
crop_grade_horse_count    5692 non-null int64
crop_grade_win_count      5692 non-null int64
dtypes: int64(4), object(2)
memory usage: 311.3+ KB


In [21]:
# nullチェック
df_mare[df_mare.isnull().any(1)]

,id,name,crop_count,crop_win_count,crop_grade_horse_count,crop_grade_win_count


In [22]:
df_mare = df_mare[['name', 'crop_win_count', 'crop_grade_horse_count', 'crop_grade_win_count']]

In [23]:
df = pd.merge(df, df_mare, left_on='mare', right_on='name', how='left', suffixes=('', '_mare'))

In [24]:
df.head()

,name,sex,stable,trainer_id,sire,mare,bms,breeder,breeder_id,prize,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year,name_mare,crop_win_count,crop_grade_horse_count,crop_grade_win_count
0,ラッキーライラック,牝,松永幹夫,01092,オルフェーヴル,ライラックスアンドレース,Flower Alley,ノーザンファーム,373126,20460.6,栗毛,480.0,5,4,92,ライラックスアンドレース,7.0,1.0,3.0
1,アーモンドアイ,牝,国枝栄,00399,ロードカナロア,フサイチパンドラ,サンデーサイレンス,ノーザンファーム,373126,17595.8,鹿毛,472.0,4,3,68,フサイチパンドラ,10.0,1.0,2.0
2,エポカドーロ,牡,藤原英昭,01055,オルフェーヴル,ダイワパッション,フォーティナイナー,田上徹,810098,17421.2,黒鹿毛,484.0,5,3,45,ダイワパッション,11.0,1.0,1.0
3,ダノンプレミアム,牡,中内田充,01137,ディープインパクト,インディアナギャル,Intikhab,ケイアイファーム,253025,16651.3,青鹿毛,476.0,4,4,92,インディアナギャル,7.0,1.0,3.0
4,ステルヴィオ,牡,木村哲也,01126,ロードカナロア,ラルケット,ファルブラヴ,ノーザンファーム,373126,13669.0,鹿毛,462.0,6,3,14,ラルケット,5.0,1.0,1.0


In [25]:
df.drop(['mare', 'name_mare'], axis=1, inplace=True)

In [26]:
df_trainer = pd.read_csv(f'{CSV_DIR}/trainer_leading.{TARGET_YEAR}.csv', index_col=[0])

In [27]:
df_trainer.head()

,id,name,stable,win_count,second_place_count,third_place_count,unplaced_count,grade_race_count,grade_race_win,stakes_race_count,...,general_race_count,general_race_win,turf_race_count,turf_race_win,dart_race_count,dart_race_win,win_ratio,in_second_place_ratio,in_third_place_ratio,prize
0,1055,藤原英昭,栗東,29,19,8,40,19,3,29,...,48,13,74,22,18,7,0.302,0.500,0.583,68888.9
1,1053,角居勝彦,栗東,20,17,10,66,13,2,42,...,58,15,89,16,22,4,0.177,0.327,0.416,35927.1
2,438,安田隆行,栗東,17,17,7,81,6,0,18,...,98,15,55,6,65,11,0.139,0.279,0.336,23462.9
3,1075,矢作芳人,栗東,17,7,18,121,13,1,53,...,97,9,97,12,63,5,0.104,0.147,0.258,37576.3
4,1126,木村哲也,美浦,15,7,5,40,11,2,12,...,44,10,50,13,15,1,0.224,0.328,0.403,27403.7


In [28]:
# nullチェック
df_trainer[df_trainer.isnull().any(1)]

,id,name,stable,win_count,second_place_count,third_place_count,unplaced_count,grade_race_count,grade_race_win,stakes_race_count,...,general_race_count,general_race_win,turf_race_count,turf_race_win,dart_race_count,dart_race_win,win_ratio,in_second_place_ratio,in_third_place_ratio,prize


In [29]:
df_trainer = df_trainer[['id', 'name', 'stable', 'win_count', 'prize']]

In [30]:
df_trainer.head()

,id,name,stable,win_count,prize
0,1055,藤原英昭,栗東,29,68888.9
1,1053,角居勝彦,栗東,20,35927.1
2,438,安田隆行,栗東,17,23462.9
3,1075,矢作芳人,栗東,17,37576.3
4,1126,木村哲也,美浦,15,27403.7


In [31]:
# Convert id to zero padding id
df_trainer['id'] = df_trainer['id'].map(lambda x:('00000' + str(x))[-5:])

In [32]:
df_trainer.head()

,id,name,stable,win_count,prize
0,01055,藤原英昭,栗東,29,68888.9
1,01053,角居勝彦,栗東,20,35927.1
2,00438,安田隆行,栗東,17,23462.9
3,01075,矢作芳人,栗東,17,37576.3
4,01126,木村哲也,美浦,15,27403.7


In [33]:
df = pd.merge(df, df_trainer, left_on='trainer_id', right_on='id', how='left', suffixes=('', '_trainer'))

In [34]:
df.head()

,name,sex,stable,trainer_id,sire,bms,breeder,breeder_id,prize,hair,...,win_count,birth_date_from_beginning_of_year,crop_win_count,crop_grade_horse_count,crop_grade_win_count,id,name_trainer,stable_trainer,win_count_trainer,prize_trainer
0,ラッキーライラック,牝,松永幹夫,01092,オルフェーヴル,Flower Alley,ノーザンファーム,373126,20460.6,栗毛,...,4,92,7.0,1.0,3.0,01092,松永幹夫,栗東,7.0,21590.4
1,アーモンドアイ,牝,国枝栄,00399,ロードカナロア,サンデーサイレンス,ノーザンファーム,373126,17595.8,鹿毛,...,3,68,10.0,1.0,2.0,00399,国枝栄,美浦,10.0,41764.5
2,エポカドーロ,牡,藤原英昭,01055,オルフェーヴル,フォーティナイナー,田上徹,810098,17421.2,黒鹿毛,...,3,45,11.0,1.0,1.0,01055,藤原英昭,栗東,29.0,68888.9
3,ダノンプレミアム,牡,中内田充,01137,ディープインパクト,Intikhab,ケイアイファーム,253025,16651.3,青鹿毛,...,4,92,7.0,1.0,3.0,01137,中内田充正,栗東,14.0,29130.2
4,ステルヴィオ,牡,木村哲也,01126,ロードカナロア,ファルブラヴ,ノーザンファーム,373126,13669.0,鹿毛,...,3,14,5.0,1.0,1.0,01126,木村哲也,美浦,15.0,27403.7


In [35]:
df.drop(['stable', 'trainer_id', 'id'], axis=1, inplace=True)

In [36]:
df.head()

,name,sex,sire,bms,breeder,breeder_id,prize,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year,crop_win_count,crop_grade_horse_count,crop_grade_win_count,name_trainer,stable_trainer,win_count_trainer,prize_trainer
0,ラッキーライラック,牝,オルフェーヴル,Flower Alley,ノーザンファーム,373126,20460.6,栗毛,480.0,5,4,92,7.0,1.0,3.0,松永幹夫,栗東,7.0,21590.4
1,アーモンドアイ,牝,ロードカナロア,サンデーサイレンス,ノーザンファーム,373126,17595.8,鹿毛,472.0,4,3,68,10.0,1.0,2.0,国枝栄,美浦,10.0,41764.5
2,エポカドーロ,牡,オルフェーヴル,フォーティナイナー,田上徹,810098,17421.2,黒鹿毛,484.0,5,3,45,11.0,1.0,1.0,藤原英昭,栗東,29.0,68888.9
3,ダノンプレミアム,牡,ディープインパクト,Intikhab,ケイアイファーム,253025,16651.3,青鹿毛,476.0,4,4,92,7.0,1.0,3.0,中内田充正,栗東,14.0,29130.2
4,ステルヴィオ,牡,ロードカナロア,ファルブラヴ,ノーザンファーム,373126,13669.0,鹿毛,462.0,6,3,14,5.0,1.0,1.0,木村哲也,美浦,15.0,27403.7


In [37]:
df_breeder_link = pd.read_csv(f'{CSV_DIR}/breeder_id.{TARGET_YEAR}.csv', index_col=[0])

In [38]:
df_breeder_link.head()

,id,alt_id,name
0,373126,b_0kejpXTUlkM,ノーザンファーム
1,810098,b_8ERFdXTEtGN,田上徹
2,253025,b_0kaVpXTElUM,ケイアイファーム
3,811540,b_8ERFhnTUF1d,ダーレー・ジャパン・ファーム
4,341126,b_0keRhXTUlkM,社台コーポレーション白老ファーム


In [39]:
df = pd.merge(df, df_breeder_link, left_on='breeder_id', right_on='id', how='left', suffixes=('', '_b'))

In [40]:
df.head()

,name,sex,sire,bms,breeder,breeder_id,prize,hair,debut_weight,race_count,...,crop_win_count,crop_grade_horse_count,crop_grade_win_count,name_trainer,stable_trainer,win_count_trainer,prize_trainer,id,alt_id,name_b
0,ラッキーライラック,牝,オルフェーヴル,Flower Alley,ノーザンファーム,373126,20460.6,栗毛,480.0,5,...,7.0,1.0,3.0,松永幹夫,栗東,7.0,21590.4,373126,b_0kejpXTUlkM,ノーザンファーム
1,アーモンドアイ,牝,ロードカナロア,サンデーサイレンス,ノーザンファーム,373126,17595.8,鹿毛,472.0,4,...,10.0,1.0,2.0,国枝栄,美浦,10.0,41764.5,373126,b_0kejpXTUlkM,ノーザンファーム
2,エポカドーロ,牡,オルフェーヴル,フォーティナイナー,田上徹,810098,17421.2,黒鹿毛,484.0,5,...,11.0,1.0,1.0,藤原英昭,栗東,29.0,68888.9,810098,b_8ERFdXTEtGN,田上徹
3,ダノンプレミアム,牡,ディープインパクト,Intikhab,ケイアイファーム,253025,16651.3,青鹿毛,476.0,4,...,7.0,1.0,3.0,中内田充正,栗東,14.0,29130.2,253025,b_0kaVpXTElUM,ケイアイファーム
4,ステルヴィオ,牡,ロードカナロア,ファルブラヴ,ノーザンファーム,373126,13669.0,鹿毛,462.0,6,...,5.0,1.0,1.0,木村哲也,美浦,15.0,27403.7,373126,b_0kejpXTUlkM,ノーザンファーム


In [41]:
df.drop(['breeder', 'breeder_id', 'id'], axis=1, inplace=True)

In [42]:
df_breeder = pd.read_csv(f'{CSV_DIR}/breeder_leading.{TARGET_YEAR}.csv', index_col=[0])

In [43]:
df_breeder.head()

,id,name,win_count,second_place_count,third_place_count,unplaced_count,grade_race_count,grade_race_win,stakes_race_count,stakes_race_win,general_race_count,general_race_win,turf_race_count,turf_race_win,dart_race_count,dart_race_win,win_ratio,in_second_place_ratio,in_third_place_ratio,prize
0,b_0kejpXTUlkM,ノーザンファーム,184,158,159,1029,162,16,430,61,938,107,1004,131,496,52,0.120,0.224,0.327,445896.6
1,b_0kerpXTUlkM,社台ファーム,107,109,96,986,73,4,266,22,959,81,660,50,608,54,0.082,0.166,0.240,213065.1
2,b_0keRhXTUlkM,社台コーポレーション白老ファーム,39,27,36,263,37,2,89,7,239,30,214,19,139,17,0.107,0.181,0.279,69328.1
3,b_8ERFhnTUF1d,ダーレー・ジャパン・ファーム,28,16,13,179,7,3,46,5,183,20,113,16,119,12,0.119,0.186,0.242,54057.3
4,b_8EVFpXTUlEM,下河辺牧場,21,16,17,194,11,1,55,6,182,14,112,10,129,10,0.085,0.149,0.218,41950.3


In [44]:
# nullチェック
df_breeder[df_breeder.isnull().any(1)]

,id,name,win_count,second_place_count,third_place_count,unplaced_count,grade_race_count,grade_race_win,stakes_race_count,stakes_race_win,general_race_count,general_race_win,turf_race_count,turf_race_win,dart_race_count,dart_race_win,win_ratio,in_second_place_ratio,in_third_place_ratio,prize


In [45]:
df_breeder = df_breeder[['id', 'name', 'win_count', 'prize']]

In [46]:
df = pd.merge(df, df_breeder, left_on='alt_id', right_on='id', how='left', suffixes=('', '_breeder'))

In [47]:
df.head()

,name,sex,sire,bms,prize,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year,...,name_trainer,stable_trainer,win_count_trainer,prize_trainer,alt_id,name_b,id,name_breeder,win_count_breeder,prize_breeder
0,ラッキーライラック,牝,オルフェーヴル,Flower Alley,20460.6,栗毛,480.0,5,4,92,...,松永幹夫,栗東,7.0,21590.4,b_0kejpXTUlkM,ノーザンファーム,b_0kejpXTUlkM,ノーザンファーム,184.0,445896.6
1,アーモンドアイ,牝,ロードカナロア,サンデーサイレンス,17595.8,鹿毛,472.0,4,3,68,...,国枝栄,美浦,10.0,41764.5,b_0kejpXTUlkM,ノーザンファーム,b_0kejpXTUlkM,ノーザンファーム,184.0,445896.6
2,エポカドーロ,牡,オルフェーヴル,フォーティナイナー,17421.2,黒鹿毛,484.0,5,3,45,...,藤原英昭,栗東,29.0,68888.9,b_8ERFdXTEtGN,田上徹,b_8ERFdXTEtGN,田上徹,4.0,18126.2
3,ダノンプレミアム,牡,ディープインパクト,Intikhab,16651.3,青鹿毛,476.0,4,4,92,...,中内田充正,栗東,14.0,29130.2,b_0kaVpXTElUM,ケイアイファーム,b_0kaVpXTElUM,ケイアイファーム,9.0,26639.7
4,ステルヴィオ,牡,ロードカナロア,ファルブラヴ,13669.0,鹿毛,462.0,6,3,14,...,木村哲也,美浦,15.0,27403.7,b_0kejpXTUlkM,ノーザンファーム,b_0kejpXTUlkM,ノーザンファーム,184.0,445896.6


In [48]:
df.drop(['alt_id', 'name_b', 'id'], axis=1, inplace=True)

In [49]:
df.head()

,name,sex,sire,bms,prize,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year,crop_win_count,crop_grade_horse_count,crop_grade_win_count,name_trainer,stable_trainer,win_count_trainer,prize_trainer,name_breeder,win_count_breeder,prize_breeder
0,ラッキーライラック,牝,オルフェーヴル,Flower Alley,20460.6,栗毛,480.0,5,4,92,7.0,1.0,3.0,松永幹夫,栗東,7.0,21590.4,ノーザンファーム,184.0,445896.6
1,アーモンドアイ,牝,ロードカナロア,サンデーサイレンス,17595.8,鹿毛,472.0,4,3,68,10.0,1.0,2.0,国枝栄,美浦,10.0,41764.5,ノーザンファーム,184.0,445896.6
2,エポカドーロ,牡,オルフェーヴル,フォーティナイナー,17421.2,黒鹿毛,484.0,5,3,45,11.0,1.0,1.0,藤原英昭,栗東,29.0,68888.9,田上徹,4.0,18126.2
3,ダノンプレミアム,牡,ディープインパクト,Intikhab,16651.3,青鹿毛,476.0,4,4,92,7.0,1.0,3.0,中内田充正,栗東,14.0,29130.2,ケイアイファーム,9.0,26639.7
4,ステルヴィオ,牡,ロードカナロア,ファルブラヴ,13669.0,鹿毛,462.0,6,3,14,5.0,1.0,1.0,木村哲也,美浦,15.0,27403.7,ノーザンファーム,184.0,445896.6


In [50]:
df_sire = pd.read_csv(f'{CSV_DIR}/sire_leading.{TARGET_YEAR}.csv', index_col=[0])

In [51]:
df_sire.head()

,id,name,race_horse_count,win_horse_count,race_count,win_count,grade_race_count,grade_race_win,stakes_race_count,stakes_race_win,...,general_race_win,turf_race_count,turf_race_win,dart_race_count,dart_race_win,win_ratio,earning_index,prize,avg_turf_distance,avg_dart_distance
0,2002100816,ディープインパクト,319,63,667,71,88,7,203,28,...,36,565,68,85,3,0.197,1.87,1860.3,1860.3,1800.0
1,2001103038,ハーツクライ,238,46,540,50,26,4,126,13,...,33,340,39,185,10,0.193,1.45,1907.7,1907.7,1780.0
2,2001103460,キングカメハメハ,215,40,466,44,56,3,136,6,...,35,200,18,249,24,0.186,1.56,1722.2,1722.2,1795.8
3,1994108729,ステイゴールド,195,32,456,34,37,4,138,15,...,15,349,28,85,4,0.164,1.70,2032.1,2032.1,1675.0
4,2001103114,ダイワメジャー,193,32,426,33,33,2,118,9,...,22,262,20,159,13,0.166,1.32,1520.0,1520.0,1557.7


In [52]:
# nullチェック
df_sire[df_sire.isnull().any(1)]

,id,name,race_horse_count,win_horse_count,race_count,win_count,grade_race_count,grade_race_win,stakes_race_count,stakes_race_win,...,general_race_win,turf_race_count,turf_race_win,dart_race_count,dart_race_win,win_ratio,earning_index,prize,avg_turf_distance,avg_dart_distance


In [53]:
df_sire = df_sire[['name', 'win_ratio', 'earning_index', 'prize']]

In [54]:
df = pd.merge(df, df_sire, left_on='sire', right_on='name', how='left', suffixes=('', '_sire'))

In [55]:
df.head()

,name,sex,sire,bms,prize,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year,...,stable_trainer,win_count_trainer,prize_trainer,name_breeder,win_count_breeder,prize_breeder,name_sire,win_ratio,earning_index,prize_sire
0,ラッキーライラック,牝,オルフェーヴル,Flower Alley,20460.6,栗毛,480.0,5,4,92,...,栗東,7.0,21590.4,ノーザンファーム,184.0,445896.6,オルフェーヴル,0.139,1.23,1781.8
1,アーモンドアイ,牝,ロードカナロア,サンデーサイレンス,17595.8,鹿毛,472.0,4,3,68,...,美浦,10.0,41764.5,ノーザンファーム,184.0,445896.6,ロードカナロア,0.198,1.37,1482.4
2,エポカドーロ,牡,オルフェーヴル,フォーティナイナー,17421.2,黒鹿毛,484.0,5,3,45,...,栗東,29.0,68888.9,田上徹,4.0,18126.2,オルフェーヴル,0.139,1.23,1781.8
3,ダノンプレミアム,牡,ディープインパクト,Intikhab,16651.3,青鹿毛,476.0,4,4,92,...,栗東,14.0,29130.2,ケイアイファーム,9.0,26639.7,ディープインパクト,0.197,1.87,1860.3
4,ステルヴィオ,牡,ロードカナロア,ファルブラヴ,13669.0,鹿毛,462.0,6,3,14,...,美浦,15.0,27403.7,ノーザンファーム,184.0,445896.6,ロードカナロア,0.198,1.37,1482.4


In [56]:
df.drop(['sire'], axis=1, inplace=True)

In [57]:
df_bms = pd.read_csv(f'{CSV_DIR}/bms_leading.{TARGET_YEAR}.csv', index_col=[0])

In [58]:
df_bms.head()

,id,name,race_horse_count,win_horse_count,race_count,win_count,grade_race_count,grade_race_win,stakes_race_count,stakes_race_win,general_race_count,general_race_win,turf_race_count,turf_race_win,dart_race_count,dart_race_win,win_ratio,earning_index,avg_turf_distance,avg_dart_distance
0,000a00033a,サンデーサイレンス,429,68,947,71,67,4,211,18,669,49,425,33,494,34,0.159,1.26,1781.8,1611.8
1,1998101516,アグネスタキオン,211,28,475,30,24,2,66,3,385,25,226,13,235,16,0.133,1.05,1815.4,1450.0
2,1998110135,クロフネ,191,26,407,28,23,1,69,10,315,17,161,16,232,12,0.136,1.11,1875.0,1691.7
3,1993109188,ダンスインザダーク,171,26,421,29,19,1,99,8,303,20,193,17,215,12,0.152,1.10,1658.8,1500.0
4,000a000082,ブライアンズタイム,187,22,431,24,11,2,86,5,334,17,137,10,287,14,0.118,0.89,1860.0,1507.1


In [59]:
# nullチェック
df_bms[df_bms.isnull().any(1)]

,id,name,race_horse_count,win_horse_count,race_count,win_count,grade_race_count,grade_race_win,stakes_race_count,stakes_race_win,general_race_count,general_race_win,turf_race_count,turf_race_win,dart_race_count,dart_race_win,win_ratio,earning_index,avg_turf_distance,avg_dart_distance


In [60]:
df_bms = df_bms[['name', 'win_ratio', 'earning_index']]

In [61]:
df = pd.merge(df, df_bms, left_on='bms', right_on='name', how='left', suffixes=('', '_bms'))

In [62]:
df.head()

,name,sex,bms,prize,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year,crop_win_count,...,name_breeder,win_count_breeder,prize_breeder,name_sire,win_ratio,earning_index,prize_sire,name_bms,win_ratio_bms,earning_index_bms
0,ラッキーライラック,牝,Flower Alley,20460.6,栗毛,480.0,5,4,92,7.0,...,ノーザンファーム,184.0,445896.6,オルフェーヴル,0.139,1.23,1781.8,Flower Alley,1.000,18.62
1,アーモンドアイ,牝,サンデーサイレンス,17595.8,鹿毛,472.0,4,3,68,10.0,...,ノーザンファーム,184.0,445896.6,ロードカナロア,0.198,1.37,1482.4,サンデーサイレンス,0.159,1.26
2,エポカドーロ,牡,フォーティナイナー,17421.2,黒鹿毛,484.0,5,3,45,11.0,...,田上徹,4.0,18126.2,オルフェーヴル,0.139,1.23,1781.8,フォーティナイナー,0.203,2.02
3,ダノンプレミアム,牡,Intikhab,16651.3,青鹿毛,476.0,4,4,92,7.0,...,ケイアイファーム,9.0,26639.7,ディープインパクト,0.197,1.87,1860.3,Intikhab,0.200,6.02
4,ステルヴィオ,牡,ファルブラヴ,13669.0,鹿毛,462.0,6,3,14,5.0,...,ノーザンファーム,184.0,445896.6,ロードカナロア,0.198,1.37,1482.4,ファルブラヴ,0.346,2.61


In [63]:
df.drop(['bms'], axis=1, inplace=True)

In [64]:
df.head()

,name,sex,prize,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year,crop_win_count,crop_grade_horse_count,...,name_breeder,win_count_breeder,prize_breeder,name_sire,win_ratio,earning_index,prize_sire,name_bms,win_ratio_bms,earning_index_bms
0,ラッキーライラック,牝,20460.6,栗毛,480.0,5,4,92,7.0,1.0,...,ノーザンファーム,184.0,445896.6,オルフェーヴル,0.139,1.23,1781.8,Flower Alley,1.000,18.62
1,アーモンドアイ,牝,17595.8,鹿毛,472.0,4,3,68,10.0,1.0,...,ノーザンファーム,184.0,445896.6,ロードカナロア,0.198,1.37,1482.4,サンデーサイレンス,0.159,1.26
2,エポカドーロ,牡,17421.2,黒鹿毛,484.0,5,3,45,11.0,1.0,...,田上徹,4.0,18126.2,オルフェーヴル,0.139,1.23,1781.8,フォーティナイナー,0.203,2.02
3,ダノンプレミアム,牡,16651.3,青鹿毛,476.0,4,4,92,7.0,1.0,...,ケイアイファーム,9.0,26639.7,ディープインパクト,0.197,1.87,1860.3,Intikhab,0.200,6.02
4,ステルヴィオ,牡,13669.0,鹿毛,462.0,6,3,14,5.0,1.0,...,ノーザンファーム,184.0,445896.6,ロードカナロア,0.198,1.37,1482.4,ファルブラヴ,0.346,2.61


In [65]:
# nullチェック （基本的に地方の調教師やオーナー）
df[df.isnull().any(1)]

,name,sex,prize,hair,debut_weight,race_count,win_count,birth_date_from_beginning_of_year,crop_win_count,crop_grade_horse_count,...,name_breeder,win_count_breeder,prize_breeder,name_sire,win_ratio,earning_index,prize_sire,name_bms,win_ratio_bms,earning_index_bms
32,ヤマノファイト,牡,4235.0,鹿毛,500.0,12,7,59,20.0,0.0,...,NaN,NaN,NaN,エスポワールシチー,0.125,0.47,0.0,フォーティナイナー,0.203,2.02
43,ストロングハート,牝,3735.0,栗毛,NaN,10,3,141,5.0,1.0,...,グランド牧場,10.0,16474.2,サウスヴィグラス,0.189,0.77,0.0,スマートボーイ,0.000,0.28
48,グラヴィオーラ,牝,3502.5,栗毛,444.0,11,4,62,10.0,0.0,...,ハクツ牧場,2.0,2075.0,サウスヴィグラス,0.189,0.77,0.0,タニノギムレット,0.111,0.97
52,プロミストリープ,牝,3420.0,栗毛,478.0,4,3,36,22.0,0.0,...,ノーザンファーム,184.0,445896.6,ヘニーヒューズ,0.193,0.80,1600.0,フジキセキ,0.114,0.71
58,レグルドール,牝,3188.2,鹿毛,456.0,7,2,78,23.0,0.0,...,NaN,NaN,NaN,アドマイヤマックス,0.000,0.22,0.0,Yonaguska,0.000,0.08
63,ハセノパイロ,牡,3087.5,鹿毛,517.0,8,3,138,18.0,0.0,...,広中稔,0.0,130.0,パイロ,0.143,0.91,1400.0,ティンバーカントリー,0.143,0.76
78,スプリングマン,牡,2752.5,黒鹿毛,478.0,11,3,111,31.0,0.0,...,斉藤政志,0.0,135.0,ローズキングダム,0.038,0.42,0.0,NaN,NaN,NaN
80,リュウノユキナ,牡,2691.3,芦毛,464.0,11,3,135,9.0,0.0,...,藤川ファーム,0.0,740.0,ヴァーミリアン,0.091,0.67,1400.0,クロフネ,0.136,1.11
85,ゴールドパテック,牝,2575.0,鹿毛,477.0,10,4,95,5.0,0.0,...,宝寄山忠則,1.0,1875.2,ゴールドアリュール,0.141,0.83,1200.0,ブライアンズタイム,0.118,0.89
90,クロスケ,牡,2461.5,青鹿毛,443.0,11,5,101,5.0,0.0,...,大狩部牧場,1.0,830.0,キャプテントゥーレ,0.067,0.31,0.0,ブライアンズタイム,0.118,0.89


In [66]:
len(df[df.isnull().any(1)])

1979

In [67]:
# null以外
df = df[~df.isnull().any(1)]

In [68]:
len(df)

3923

In [69]:
df.head()

,name,sex,prize,hair,debut_weight,race_count,birth_date_from_beginning_of_year,crop_win_count,crop_grade_horse_count,crop_grade_win_count,...,name_breeder,win_count_breeder,prize_breeder,name_sire,win_ratio,earning_index,prize_sire,name_bms,win_ratio_bms,earning_index_bms
0,ラッキーライラック,牝,20460.6,栗毛,480.0,5,92,7.0,1.0,3.0,...,ノーザンファーム,184.0,445896.6,オルフェーヴル,0.139,1.23,1781.8,Flower Alley,1.000,18.62
1,アーモンドアイ,牝,17595.8,鹿毛,472.0,4,68,10.0,1.0,2.0,...,ノーザンファーム,184.0,445896.6,ロードカナロア,0.198,1.37,1482.4,サンデーサイレンス,0.159,1.26
2,エポカドーロ,牡,17421.2,黒鹿毛,484.0,5,45,11.0,1.0,1.0,...,田上徹,4.0,18126.2,オルフェーヴル,0.139,1.23,1781.8,フォーティナイナー,0.203,2.02
3,ダノンプレミアム,牡,16651.3,青鹿毛,476.0,4,92,7.0,1.0,3.0,...,ケイアイファーム,9.0,26639.7,ディープインパクト,0.197,1.87,1860.3,Intikhab,0.200,6.02
4,ステルヴィオ,牡,13669.0,鹿毛,462.0,6,14,5.0,1.0,1.0,...,ノーザンファーム,184.0,445896.6,ロードカナロア,0.198,1.37,1482.4,ファルブラヴ,0.346,2.61


In [69]:
df.to_pickle('merged_data.pkl')